<a href="https://colab.research.google.com/github/The-cheater/Deep_Learning_Models/blob/main/pak_si_ju_ga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 1. Load image paths from a single folder (control or park)
def load_class_images(class_dir, label_name):
    left_paths, right_paths, labels = [], [], []
    for fname in sorted(os.listdir(class_dir)):
        if "left" in fname:
            left_path = os.path.join(class_dir, fname)
            right_path = left_path.replace("left", "right")
            if os.path.exists(right_path):
                left_paths.append(left_path)
                right_paths.append(right_path)
                labels.append(label_name)
    return left_paths, right_paths, labels

# 2. Collect all GAF and MTF paths
gaf_left_control, gaf_right_control, labels_control = load_class_images("/content/gaf_images/train/control", "control")
gaf_left_park, gaf_right_park, labels_park = load_class_images("/content/gaf_images/train/park", "park")

mtf_left_control, mtf_right_control, _ = load_class_images("/content/mtf_images/train/control", "control")
mtf_left_park, mtf_right_park, _ = load_class_images("/content/mtf_images/train/park", "park")

# Combine all
gaf_left = gaf_left_control + gaf_left_park
gaf_right = gaf_right_control + gaf_right_park
mtf_left = mtf_left_control + mtf_left_park
mtf_right = mtf_right_control + mtf_right_park
labels = labels_control + labels_park

# 3. Encode Labels
le = LabelEncoder()
y = le.fit_transform(labels)

# 4. Train/Val Split
gaf_l_train, gaf_l_val, gaf_r_train, gaf_r_val, mtf_l_train, mtf_l_val, mtf_r_train, mtf_r_val, y_train, y_val = train_test_split(
    gaf_left, gaf_right, mtf_left, mtf_right, y, test_size=0.2, random_state=42
)

# 5. Image Loader
def load_images(paths, size=(64, 64)):
    images = []
    for p in paths:
        img = load_img(p, target_size=size)
        img = img_to_array(img) / 255.0
        images.append(img)
    return np.array(images)

X_gaf_l = load_images(gaf_l_train)
X_gaf_r = load_images(gaf_r_train)
X_mtf_l = load_images(mtf_l_train)
X_mtf_r = load_images(mtf_r_train)

X_gaf_val_l = load_images(gaf_l_val)
X_gaf_val_r = load_images(gaf_r_val)
X_mtf_val_l = load_images(mtf_l_val)
X_mtf_val_r = load_images(mtf_r_val)

# 6. CNN Block
def cnn_block(input_tensor):
    x = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001))(input_tensor)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    return x

# 7. Model Inputs
input_gaf_l = Input(shape=(64, 64, 3))
input_gaf_r = Input(shape=(64, 64, 3))
input_mtf_l = Input(shape=(64, 64, 3))
input_mtf_r = Input(shape=(64, 64, 3))

# 8. Apply CNN Blocks
gaf_l_feat = cnn_block(input_gaf_l)
gaf_r_feat = cnn_block(input_gaf_r)
mtf_l_feat = cnn_block(input_mtf_l)
mtf_r_feat = cnn_block(input_mtf_r)

# 9. Concatenate & Dense Layers
combined = concatenate([gaf_l_feat, gaf_r_feat, mtf_l_feat, mtf_r_feat])
x = Dense(256, activation='relu')(combined)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[input_gaf_l, input_gaf_r, input_mtf_l, input_mtf_r], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 10. Train
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]

history = model.fit(
    [X_gaf_l, X_gaf_r, X_mtf_l, X_mtf_r],
    y_train,
    validation_data=([X_gaf_val_l, X_gaf_val_r, X_mtf_val_l, X_mtf_val_r], y_val),
    epochs=30,
    batch_size=16,
    callbacks=callbacks
)


Epoch 1/30
14/14 [==============================] - 16s 853ms/step - loss: 15.2603 - accuracy: 0.5982 - val_loss: 0.9219 - val_accuracy: 0.6727
Epoch 2/30
14/14 [==============================] - 9s 647ms/step - loss: 6.9821 - accuracy: 0.7306 - val_loss: 0.9846 - val_accuracy: 0.5273
Epoch 3/30
14/14 [==============================] - 10s 714ms/step - loss: 1.4447 - accuracy: 0.8493 - val_loss: 7.8402 - val_accuracy: 0.6727
Epoch 4/30
14/14 [==============================] - 10s 752ms/step - loss: 1.5293 - accuracy: 0.9132 - val_loss: 21.4041 - val_accuracy: 0.6909
Epoch 5/30
14/14 [==============================] - 9s 649ms/step - loss: 1.0318 - accuracy: 0.9041 - val_loss: 32.5244 - val_accuracy: 0.6909
Epoch 6/30
14/14 [==============================] - 10s 711ms/step - loss: 0.7826 - accuracy: 0.9269 - val_loss: 42.2024 - val_accuracy: 0.6909


In [18]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import files
import numpy as np

# Step 1: Load trained multimodal model
model = load_model("best_model.keras")  # Update if your model has a different name

# Step 2: Helper function to preprocess image
def preprocess_image(img_path, target_size=(64, 64)):
    img = load_img(img_path, target_size=target_size)
    img = img_to_array(img) / 255.0
    return np.expand_dims(img, axis=0)

# Step 3: Upload 4 images (you'll be prompted)
print("📤 Please upload the following 4 files:\n1. GAF Left\n2. GAF Right\n3. MTF Left\n4. MTF Right")

uploaded = files.upload()

# Step 4: Extract and preprocess each uploaded file
uploaded_files = list(uploaded.keys())
if len(uploaded_files) != 4:
    raise ValueError("❌ Please upload exactly 4 images: GAF Left, GAF Right, MTF Left, MTF Right")

# Sort or manually map to ensure correct order
# (Assumes you name your files like: user_left_gaf.png, user_right_gaf.png, etc.)
gaf_left_img = preprocess_image(uploaded_files[0])
gaf_right_img = preprocess_image(uploaded_files[1])
mtf_left_img = preprocess_image(uploaded_files[2])
mtf_right_img = preprocess_image(uploaded_files[3])

# Step 5: Predict
prediction = model.predict([gaf_left_img, gaf_right_img, mtf_left_img, mtf_right_img])[0][0]
label = "Parkinson's Disease" if prediction > 0.5 else "Control (Normal)"
confidence = prediction if prediction > 0.5 else 1 - prediction

# Step 6: Output result
print(f"\n✅ Prediction: {label} with confidence: {confidence:.2f}")


📤 Please upload the following 4 files:
1. GAF Left
2. GAF Right
3. MTF Left
4. MTF Right


Saving control01_left_gaf.png to control01_left_gaf (1).png
Saving control01_left_mtf.png to control01_left_mtf (1).png
Saving control01_right_gaf.png to control01_right_gaf (1).png
Saving control01_right_mtf.png to control01_right_mtf (1).png
1/1 [==============================] - 0s 237ms/step

✅ Prediction: Parkinson's Disease with confidence: 0.98
